In [ ]:
!pip install optuna -q
!pip install librosa -q
!pip install eli5 -q
!pip install catboost -q
!pip install tensorflow -q
!pip install joblib -q
!pip install scipy -q
!pip install h5py scikit-optimize -q
!pip3 install wget -q

In [ ]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

In [ ]:
import shutil
import os

DATASET_LOCATION = "/kaggle/working/dataset.csv"

shutil.rmtree("Music", ignore_errors=True)
try:
    os.remove(DATASET_LOCATION)
except:
    print("Oops, no file there")

In [ ]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

print(f"{bcolors.HEADER}Initialized colours !{bcolors.ENDC}")
print(f"{bcolors.OKBLUE}Initialized colours !{bcolors.ENDC}")
print(f"{bcolors.OKCYAN}Initialized colours !{bcolors.ENDC}")
print(f"{bcolors.OKGREEN}Initialized colours !{bcolors.ENDC}")
print(f"{bcolors.WARNING}Initialized colours !{bcolors.ENDC}")
print(f"{bcolors.FAIL}Initialized colours !{bcolors.ENDC}")
print(f"{bcolors.ENDC}Initialized colours !{bcolors.ENDC}")
print(f"{bcolors.BOLD}Initialized colours !{bcolors.ENDC}")
print(f"{bcolors.UNDERLINE}Initialized colours !{bcolors.ENDC}")

In [ ]:
#!conda install -y gdown

#https://drive.google.com/file/d/1FQQt2mHJl6BBwsxDuERgavWE1cUkxlut/view?usp=sharing
#!gdown 1FQQt2mHJl6BBwsxDuERgavWE1cUkxlut

In [ ]:
import os
import time
import librosa
from librosa.core.audio import zero_crossings
import librosa.display
import optuna
import numpy as np
import json
from sklearn.metrics import classification_report
import IPython.display as ipd
%matplotlib inline
import matplotlib.pyplot as plt
import joblib
import sklearn
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import make_scorer , mean_squared_error, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, classification_report
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import GridSearchCV

# Skopt functions
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, DeltaYStopper,VerboseCallback,DeltaXStopper
from skopt.space import Real, Categorical, Integer

from PIL import Image
import pathlib
import csv
from xgboost import XGBClassifier, XGBRFClassifier
from xgboost import plot_tree, plot_importance
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint,LearningRateScheduler
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from xgboost import plot_importance
import catboost as cb
from catboost import CatBoostError
import eli5
from eli5.sklearn import PermutationImportance
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

#For downloading music from WEB
import urllib.request
from urllib.request import urlopen
import wget
import requests
from bs4 import BeautifulSoup

In [ ]:
# ladd = "filename,chroma_stft,chroma_cqt,chroma_cens,melspectrogram,mfcc,rmse,spectral_centroid,spectral_bandwidth,spectral_contrast,spectral_flatness,rolloff,poly_features,tonnetz,zero_crossing_rate,tempogram,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label".split(",")
dataset = pd.read_csv("/kaggle/input/formusic/dataset.csv")#,usecols=ladd)
dataset.to_csv("dataset.csv",index=False)
dataset

In [ ]:
class DownloadingMusic:
  def __init__(self, genre,url_genre, music_path):
    self.genre = genre
    self.url_genre = url_genre
    self.music_path = music_path
    self.makeGenreDir()
  

  def makeGenreDir(self):
    if not os.path.exists(self.music_path+self.genre):
      os.makedirs(self.music_path+self.genre)
    
    
  def get_music_from_release(self,url):
    #genre = "Trip-Hop"
    #url = "https://musify.club/release/eriks-iridescent-tent-peter-piper-pepperidge-farm-at-the-gates-of-dawn-detergent-2022-1679379"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    result_genre = soup.find(id="bodyContent")
    element_in_genre = result_genre.find_all("a", href=True)
    for i in element_in_genre:
      #print(i)
      if (".mp3" in i['href']):
        var = "https://musify.club"+i['href']
        songname=var.split("/")[-1]
        if( (checkDownloadedFileInCsv(songname)) and (songname not in os.listdir(self.music_path+self.genre) ) ):
          print(f"{bcolors.BOLD}Downloading: "+var + "  "+songname+"\n")
          #wget.download(f'{var}')
    
          try: # We want to catch mistakes like 503 and so on
            mp3file = urlopen(var)
            with open(self.music_path+self.genre+"/"+songname,'w+b') as output:
              output.write(mp3file.read())
    
          except OSError:
            print(f"{bcolors.FAIL}Huinia s failom {songname}")
            songname=i['download']
            if( (checkDownloadedFileInCsv(songname)) and (songname not in os.listdir(self.music_path+self.genre) ) ):
                
                try:
                    print(f"{bcolors.BOLD}Downloading: "+var + "  "+songname+"\n")
                    mp3file = urlopen(var)
                    with open(self.music_path+self.genre+"/"+songname,'w+b') as output:
                        output.write(mp3file.read())
                        
                except:
                    print(f"{bcolors.FAIL}File nedostupen !!!! {songname}")
        #print()
            else:
                print(f"{bcolors.OKGREEN}{songname} already exists"+"\n")
                    
        
  def get_list_of_pages(self,get_url):
    page = requests.get(get_url)
    soup = BeautifulSoup(page.content, "html.parser")
    result_genre = soup.find(id="result")
    element_in_genre = result_genre.find_all("a", href=True)
    pages_list = []
    pages_list.append(get_url)
    for i in element_in_genre:
      numb_of_page=get_url+"/"+i['href'].split('/')[-1]
      if ("/page" in i['href'] and (numb_of_page not in pages_list)):
        pages_list.append(numb_of_page)
    print(pages_list)
    if len(pages_list)>0:
        start = str(pages_list[1]).split('/')[-1].split('page')[-1]
        end = pages_list[-1].split('/')[-1].split('page')[-1]
        
        for i in range(int(start),int(end)+1):
          if (f"{get_url}/page{i}" in pages_list)==False:
            pages_list.append(f"{get_url}/page{i}")
          
    return (pages_list)


  def get_releases(self):
    #genre_url = "https://musify.club/genre/acid-rock-188"
    listing_of_genres = self.get_list_of_pages(self.url_genre) 
    releases_list = []
    for page_genre in listing_of_genres:
      #print(ganra)
      page = requests.get(page_genre)
      soup = BeautifulSoup(page.content, "html.parser")
      result_genre = soup.find(id="result")
      element_in_genre = result_genre.find_all("a", href=True)
      #print(element_in_genre)
      for boba in element_in_genre:
        link_to_release=boba['href']
        if (("/release/" in link_to_release) and (link_to_release not in releases_list)):
          #print(link_to_release)
          releases_list.append(link_to_release)
        #print()
    return(releases_list)

In [ ]:
def checkDownloadedFileInCsv(name):
  with open(DATASET_LOCATION, 'r') as fp:
    s = fp.read()
    if name.split(".mp3")[0] not in s:
      print(f"{bcolors.OKBLUE}There is NO {name} in dataset")
      return True
    else:
      return False

In [ ]:
def process_songs(music_path,music_genre):
  #some_path = music_path+f"/Acid Rock" #music_path+f"/{music_genre}"
  if (os.path.exists(DATASET_LOCATION)):
    some_path = music_path+f"/{music_genre}"
    #Need this to count time
    st_cpu = time.process_time()  
    st_time = time.time()
    print(f"{bcolors.HEADER}BEGINNING PROCCESSING")
    print(f"{bcolors.BOLD}-"*40)
    print(f"{bcolors.OKCYAN}PATH: {some_path}")
    #Создаем Header для файла CSV
    header = 'filename chroma_stft chroma_cqt chroma_cens melspectrogram mfcc rmse spectral_centroid spectral_bandwidth spectral_contrast spectral_flatness rolloff poly_features tonnetz zero_crossing_rate tempogram'
    for i in range(1, 21): 
        header += f' mfcc{i}'
    header += ' label'
    header = header.split()
    
    
    for path in os.listdir(some_path):
        if os.path.isfile(os.path.join(some_path, path)):
          if(checkDownloadedFileInCsv(path.split('.mp3')[0])):
            filename = path.replace(' ', '_')
            y, sr = librosa.load(some_path+"/"+path, mono=True)
            chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
            chroma_cqt = librosa.feature.chroma_cqt(y=y, sr=sr)
            chroma_cens = librosa.feature.chroma_cens(y=y, sr=sr)
            melspectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
            mfcc = librosa.feature.mfcc(y=y, sr=sr)
            rmse = librosa.feature.rms(y=y)
            spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
            spectral_flatness = librosa.feature.spectral_flatness(y=y)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            poly_features = librosa.feature.poly_features(y=y, sr=sr)
            tonnetz = librosa.feature.tonnetz(y=y, sr=sr)
            zcr = librosa.feature.zero_crossing_rate(y)
            tempogram = librosa.feature.tempogram(y=y, sr=sr)
            to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(chroma_cqt)} {np.mean(chroma_cens)} {np.mean(melspectrogram)} {np.mean(mfcc)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(spectral_contrast)} {np.mean(spectral_flatness)} {np.mean(rolloff)} {np.mean(poly_features)} {np.mean(tonnetz)} {np.mean(zcr)} {np.mean(tempogram)}'    
            for e in mfcc:
                to_append += f' {np.mean(e)}'
            to_append += f' {music_genre.replace(" ", "_")}'
            file = open(DATASET_LOCATION, 'a', newline='')
            with file:
                writer = csv.writer(file)
                writer.writerow(to_append.split())
            print(f"{bcolors.UNDERLINE}Deleting FILE\n")
            os.remove(some_path+"/"+path)
    print(f"{bcolors.OKGREEN}\nCPU TIME (min): "+ str((time.process_time() - st_cpu)/60)+"\nMethod Time (min): "+ str((time.time() - st_time)/60)+"\n")
  else:
    print(f"{bcolors.FAIL}THERE IS NO DATASET !")
    return

In [ ]:
def getALLIDS():
  URL = "https://musify.club/genre"
  page = requests.get(URL)
  IDS = []
  soup = BeautifulSoup(page.content, "html.parser")
  for ID in soup.find_all('div', id=True):  
      if("genre_" in ID.get('id')):
        IDS.append(ID.get('id'))
  return(IDS)

In [ ]:
import multiprocessing
def hang():
    while True:
        pass

In [ ]:
IDS = getALLIDS()
for id in IDS:
  print(f"{bcolors.WARNING}USING ID = {id}")
  URL = "https://musify.club/genre"
  page = requests.get(URL)
  soup = BeautifulSoup(page.content, "html.parser")
  #id = "genre_1"
  results = soup.find(id=id)
  music_path = "Music/"+id
  if not os.path.exists(music_path):
    os.makedirs(music_path)
  job_elements = results.find_all("a", href=True)
  for element in job_elements:
    music_genre = element.text[1:]
    url_genre = "https://musify.club"+element['href']
    print(f"Genre: {music_genre}")
    print(f"URL: {url_genre}")
    music = DownloadingMusic(music_genre, url_genre, music_path+"/")
    current_genre = music.get_releases()
    print(len(current_genre))
    #print(current_genre)
    for i in current_genre:
      print(f"{bcolors.ENDC}THIS RELEASE ", i)
      music.get_music_from_release("https://musify.club/"+i)
        
      if ("/kaggle/" not in DATASET_LOCATION):
          p = multiprocessing.Process(target=hang)
          p.start()
          process_songs(music_path, music_genre)
          p.terminate()
          p.join()
          p.close()
          print ('main process exiting..')
      else:  
          process_songs(music_path, music_genre)
    
    #current_genre = get_releases(url_genre)
    # print(current_genre)
    # print(len(current_genre)) #https://musify.club